In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots 
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
import scipy.stats as stats
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame

In [59]:
#Importing files

train_data = pd.read_parquet(r"C:\Users\DELL\OneDrive - GANIT BUSINESS SOLUTIONS PRIVATE LIMITED\Team2_mock\Code\Treat_train_final2.parquet")
x_test_data = pd.read_parquet(r"C:\Users\DELL\OneDrive - GANIT BUSINESS SOLUTIONS PRIVATE LIMITED\Team2_mock\Code\Treat_test_final2.parquet")
y_test_data = pd.read_csv(r"C:\Users\DELL\OneDrive - GANIT BUSINESS SOLUTIONS PRIVATE LIMITED\Team2_mock\y_test.csv")

In [60]:
#Extracting X_train, X_test, Y_train and Y_test and converting them into dataframes

X_train = train_data.drop(['tracking_id','datetime','generated_power'],axis=1)
Y_train = train_data['generated_power']
X_test  = x_test_data.drop(['tracking_id','datetime'],axis=1)
Y_test  = y_test_data.drop(['tracking_id','datetime'], axis = 1)

Y_test.columns = ['true_power']
Y_train.columns = ['generated_power']

Y_train = DataFrame(Y_train)
Y_test = DataFrame(Y_test)

In [61]:
#Scaling using minmax scaler

scaler = MinMaxScaler()
x_train = pd.DataFrame(scaler.fit_transform(X_train[X_train.columns]), columns=X_train.columns)
y_train = pd.DataFrame(scaler.fit_transform(Y_train[Y_train.columns]), columns=Y_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(X_test[X_test.columns]), columns=X_test.columns)
y_test = pd.DataFrame(scaler.fit_transform(Y_test[Y_test.columns]), columns=Y_test.columns)

In [62]:
#Polynomial Regression Code

from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

Poly_Regression = PolynomialFeatures(degree=2)
poly_var_train = Poly_Regression.fit_transform(x_train)
poly_var_test = Poly_Regression.fit_transform(x_test)

regression = linear_model.LinearRegression()
model = regression.fit(poly_var_train, y_train)

y_pred = regression.predict(poly_var_test)
y_pred =  DataFrame(y_pred)
y_pred.columns = ['gen_tpower']
y_pred
# model.score(x_test, y_test)

,gen_tpower
0,0.231022
1,0.241917
2,0.239288
3,0.298904
4,0.379442
...,...
12081,0.518114
12082,0.396794
12083,0.341487
12084,0.393959


In [63]:
#reshaping to get an array

y_hat = y_pred.values.reshape(12086,1)

In [64]:
#creating another df with both predicted and actual power columns in it

plotting = y_test.copy()
plotting['predicted'] = y_hat

In [65]:
px.line(plotting, y=['true_power', 'predicted'])

Without Normalising

In [96]:
Poly_Regression = PolynomialFeatures(degree=3)
poly_var_train2 = Poly_Regression.fit_transform(X_train)
poly_var_test2 = Poly_Regression.fit_transform(X_test)

regression2 = linear_model.LinearRegression()
model2 = regression2.fit(poly_var_train2, Y_train)

Y_pred = regression2.predict(poly_var_test2)
Y_pred =  DataFrame(Y_pred)
Y_pred.columns = ['gen_tpower1']
Y_pred


,gen_tpower1
0,2.535120
1,2.141444
2,2.324750
3,4.731791
4,5.281985
...,...
12081,7.259905
12082,4.829677
12083,6.358073
12084,5.310093


In [97]:
#reshaping to get an array

Y_hat1 = Y_pred.values.reshape(12086,1)

In [98]:
#creating another df with both predicted and actual power columns in it

plotting1 = Y_test.copy()
plotting1['predicted'] = Y_hat1

In [99]:
px.line(plotting1.head(100), y=['true_power', 'predicted'])

In [103]:
#before scaling
from sklearn.metrics import mean_squared_error, r2_score
rmse2 = np.sqrt(mean_squared_error(Y_test,Y_pred))
r22 = r2_score(Y_test,Y_pred)
print(rmse2)
print(r22)

1.0842396340303193
0.8378844385086787


In [102]:
#before scaling
mae1= metrics.mean_absolute_error(Y_test,Y_pred)
print("mae:",mae1)
mse1 = metrics.mean_squared_error(Y_test,Y_pred)
print("mse:",mse1)
rmse1 = np.sqrt(mse1)
print("rmse:",rmse1)

mae: 0.7992706701859326
mse: 1.1755755840022009
rmse: 1.0842396340303193


In [86]:
#after scaling
mae1= metrics.mean_absolute_error(y_test,y_pred)
print("mae:",mae1)
mse1 = metrics.mean_squared_error(y_test,y_pred)
print("mse:",mse1)
rmse1 = np.sqrt(mse1)
print("rmse:",rmse1)

mae: 0.14743110149411853
mse: 0.026562925450853134
rmse: 0.16298136534847515


In [70]:
#after scaling
from sklearn.metrics import mean_squared_error, r2_score
rmse2 = np.sqrt(mean_squared_error(y_test,y_pred))
r22 = r2_score(y_test,y_pred)
print(rmse2)
print(r22)

0.16298136534847515
-0.20634583733894551


Feature importance

In [1]:
# import eli5
# from eli5.sklearn import PermutationImportance

# feature_names = [i for i in poly_var_train2df.columns if poly_var_train2df[i].dtype in [np.int64]]
# perm = PermutationImportance(model, random_state=1).fit(poly_var_test2df, Y_test)
# eli5.show_weights(perm, feature_names = poly_var_test2df.columns.tolist())

In [81]:
poly_var_testdf = DataFrame(poly_var_test)
poly_var_traindf =  DataFrame(poly_var_train)
poly_var_test2df = DataFrame(poly_var_test2)
poly_var_train2df =  DataFrame(poly_var_train2)

In [84]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, poly_var_test, y_test,
                          n_repeats=2,
                          random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(
           f"{r.importances_mean[i]:.3f}"
            f" +/- {r.importances_std[i]:.3f}")

104662827742532730880.000 +/- 893379356974456832.000
55769262474805452800.000 +/- 476034413761306624.000
48016195561195700224.000 +/- 9719808.863
45351265817132056576.000 +/- 185107207465725952.000
40901390649285648384.000 +/- 163170441440944128.000
24450003099780988928.000 +/- 83447109562972160.000
16806426191608631296.000 +/- 5752832.000
16162219172627234816.000 +/- 65968241491758080.000
14436212032980520960.000 +/- 40044970949625856.000
13323249663605760000.000 +/- 53151261943929856.000
11801759837181630464.000 +/- 24300123192526848.000
10303668387006906368.000 +/- 6747654510371840.000
9791533766032154624.000 +/- 32358009799064576.000
8972761741595979776.000 +/- 60300818163585536.000
8853232254872445952.000 +/- 95196045750488064.000
4996144141739444224.000 +/- 17051686487229952.000
4313067255474461696.000 +/- 47073039630445312.000
3821707905833950208.000 +/- 8434071410322176.000
3663863219398924288.000 +/- 13816419398320384.000
3644906310258917376.000 +/- 13744932939469568.000
31134